In [185]:
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import dash
import dash_bootstrap_components as dbc

In [186]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [187]:
from pandas import DataFrame

covid_df: DataFrame = pd.read_csv('../datasets\covid.csv')
salary_ds: DataFrame = pd.read_csv('../datasets\ds_salary.csv', sep=';')
gym_df = pd.read_csv('../datasets/gym_ex.csv', sep=';')
lifestyle_df = pd.read_csv('../datasets/health_lifestyle.csv')

In [188]:
covid_df.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


In [189]:
from plotly.graph_objects import Figure

# Gráficas básicas


bar_fig: Figure = px.bar(lifestyle_df, 
                    x='Occupation', 
                    y='Sleep Duration', 
                    color='BMI Category', 
                    
                    title="Media de sueño en función de la profesión y Categoría de Peso", 
                    labels={'Occupation': 'Ocupación', 'Sleep Duration': 'Acumulación de media de Hora Dormida'},
                    barmode='group'
                    )

salary_avg_by_experience = salary_ds[salary_ds['salary_currency'] == 'USD' ]
salary_avg_by_experience = salary_ds[salary_ds['company_location'] == 'US']

salary_avg_by_experience = salary_ds.groupby('experience_level')['salary_in_usd'].mean().reset_index()

line_fig = px.line(salary_avg_by_experience, 
                    x='experience_level',  
                    y='salary_in_usd',  
                    
                    title="Media de Salario por Nivel de Experiencia en EEUU",  
                    labels={'experience_level': 'Nivel de Exp.', 'salary_in_usd': 'Salario medio(USD)'},
                    markers=True)  

scatter_fig: Figure = px.scatter(lifestyle_df, 
                                x='Age', 
                                y='Sleep Duration', 
                                
                                color='Occupation', 
                                size='Physical Activity Level',  
                                hover_name='Person ID', 
                                
                                title="Duración de Sueño vs Edad por Profesión", 
                                labels={'Sleep Duration': 'Duración de Sueño (hrs)', 'Age': 'Edad', 'Occupation': 'Ocupación',},
                                color_discrete_map={"Overweight": "orange", "Normal": "green", "Obese": "red"})

workout_type_counts = gym_df['Workout_Type'].value_counts(normalize=True) * 100
total_users = len(gym_df)
workout_type_percentage = (workout_type_counts / total_users) * 100

per_fig = px.pie(workout_type_percentage,
                values=workout_type_percentage.values,  # Los valores de porcentaje calculados
                names=workout_type_percentage.index,   # Los nombres de los tipos de entrenamiento
                title="% de usuarios en actividades",
                color=workout_type_percentage.index,
                color_discrete_map={'Yoga': 'blue', 'HIIT': 'green', 'Cardio': 'orange', 'Strength': 'red'},
                hole=0.3)  # Si deseas un gráfico de dona


In [190]:
x = [1, 2, 3, 4, 5]
y = [10, 15, 13, 17, 16]

# Gráficas estadísticas
box_fig: Figure = px.box(gym_df, 
                x='Workout_Type', 
                y='Session_Duration (hours)', 
                title="Duración de sesion por tipo de entrenamiento",  
                labels={'Session_Duration (hours)': 'Duración (horas)', 'Workout_Type': 'Tipo'},
                color='Workout_Type', 
                boxmode='group') 



hist_fig: Figure = px.histogram(gym_df, 
                                x='Session_Duration (hours)',  # Duración de la sesión en horas
                                title="Distribución de la duración de sesión",
                                labels={'Session_Duration (hours)': 'Duración (horas)', 'Workout_Type': 'Tipo'},
                                color='Workout_Type',  # Colorear por tipo de ejercicio
                                nbins=20,  # Número de bins
                                opacity=0.7,  # Opacidad para mejor visualización
                                marginal='rug'  # Añadir marcas de rug (pequeñas líneas) para indicar la distribución de los datos
                                )

hist_fig.show()

In [191]:
# Gráficas 3D
# Crear un scatter 3D
x = [1, 2, 3, 4, 5]
y = [10, 15, 13, 17, 16]

salary_ds_cleaned = salary_ds.dropna(subset=['salary_in_usd', 'experience_level', 'company_size'])

scatter_3d_fig = px.scatter_3d(salary_ds_cleaned,
                                x='salary_in_usd',  # Salario en USD
                                y='experience_level',  # Nivel de experiencia
                                z='company_size',  # Tamaño de la empresa
                                color='experience_level',  # Colorear por nivel de experiencia
                                size='salary_in_usd',  # Tamaño de los puntos según el salario
                                hover_data=['job_title'],  # Mostrar el título del trabajo al pasar el mouse
                                title="3D Visualization of Salary by Experience Level and Company Size",  # Título
                                labels={'salary_in_usd': 'Salary (USD)', 
                                        'experience_level': 'Experience Level', 
                                        'company_size': 'Company Size'},
                                opacity=0.7)

scatter_3d_fig.update_layout(
    scene=dict(
        xaxis_title='Salary (USD)',
        yaxis_title='Experience Level',
        zaxis_title='Company Size'
    ),
    margin=dict(l=0, r=0, b=0, t=40),  # Ajuste de márgenes
    showlegend=True  # Mostrar la leyenda
)



# Crear un plano con un punto de silla
X = np.linspace(-5, 5, 100)
Y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(X, Y)
Z = X**2 - Y**2  # Punto de silla

saddle_fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

saddle_fig.update_layout(title='Plano con Punto de Silla', 
                         scene=dict(
                            xaxis_title='Eje X',
                            yaxis_title='Eje Y',
                            zaxis_title='Eje Z',
                        ),
                         margin=dict(l=0, r=0, b=0, t=40),  # Ajuste de márgenes
    showlegend=True
)

In [192]:
from plotly.graph_objects import Figure
from pandas import DataFrame
import plotly.express as px

df:DataFrame = px.data.gapminder()

animation = px.scatter(df, 
            x="gdpPercap", 
            y="lifeExp", 
            color="continent", 
            log_x=True, 
            size_max=55, 
            range_x=[100,100000], 
            range_y=[25,90],
            # Solo con esto !
            animation_frame="year", 
            animation_group="country",)

In [193]:
app.layout = dbc.Container([
    html.H1("Ejemplo de Gráficas con Dash y Plotly", className="text-center mb-4"),
    
    # Selector para elegir el tipo de gráficas
    dcc.Dropdown(
        id='graph-selector',
        options=[
            {'label': 'Gráficas Básicas', 'value': 'basic'},
            {'label': 'Gráficas Estadísticas', 'value': 'stats'},
            {'label': 'Gráficas 3D', 'value': '3d'}
        ],
        value='basic',
        clearable=False,
        className='mb-4'
    ),
    
    # Área donde se mostrarán las gráficas
    html.Div(id='graphs-container')
], fluid=True, style={'height': '100vh'})  # Asegurar que el contenedor ocupe todo el alto de la ventana


In [194]:
@app.callback(
    Output('graphs-container', 'children'),
    Input('graph-selector', 'value')
)
def update_graphs(selected_value):
    if selected_value == 'basic':
        return [
            html.H2("Gráficas Básicas", className="mt-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=bar_fig), width=6),
                dbc.Col(dcc.Graph(figure=line_fig), width=6),
            ], className="mb-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=scatter_fig), width=6),
                dbc.Col(dcc.Graph(figure=per_fig), width=6),  # Espacio vacío para mantener el diseño
            ], className="mb-4"),
        ]
    elif selected_value == 'stats':
        return [
            html.H2("Gráficas Estadísticas", className="mt-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=box_fig), width=6),
                dbc.Col(dcc.Graph(figure=hist_fig), width=6),
            ], className="mb-4"),
        ]
    elif selected_value == '3d':
        return [
            html.H2("Gráficas 3D", className="mt-4"),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=scatter_3d_fig), width=6),
                dbc.Col(dcc.Graph(figure=saddle_fig), width=6),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(figure=animation), width=12),  # Figura de animación ocupando todo el ancho
            ], className="mt-4")
        ]

if __name__ == "__main__":
    app.run_server(debug=True, port=8050)


In [196]:
# Exporta las figuras a archivos HTML
bar_fig.write_html('../bar_fig.html')
line_fig.write_html('../line_fig.html')
scatter_fig.write_html('../scatter_fig.html')
per_fig.write_html('../per_fig.html')
box_fig.write_html('../box_fig.html')
hist_fig.write_html('../hist_fig.html')
scatter_3d_fig.write_html('../scatter_3d_fig.html')
saddle_fig.write_html('../saddle_fig.html')
animation.write_html('../animation_fig.html')
